[View in Colaboratory](https://colab.research.google.com/github/opooladz/ece_210_final_project/blob/master/210_final_project_classify_mod_convnet.ipynb)

In [0]:
# Import Libs 
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math
#from  scipy.ndimage import convolve1d

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Keras try model 
def try_model(model, data, name, epochs=200, verbose=1):  
  model.compile(optimizer='nadam',
            loss='categorical_crossentropy',
            metrics=[ 'accuracy'])

  model.summary()  
  history = model.fit(data['x_train'], data['y_train'], 
            epochs=epochs, batch_size=64, verbose=verbose, 
            validation_data=(data['x_val'], data['y_val']))  
  score = model.evaluate(data['x_test'], data['y_test'])
  print('Test accuracy: {}'.format(score))
  
#   out_path = 'serialized_models/{}.h5'.format(name)  
#   model.save(out_path)
  
#   with open('serialized_models/{}_history.pickle'.format(name), 'wb') as handle:
#     pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
  return history

In [0]:
# Comms Stuff Will add more later
def qammod(m,y):
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  # x = np.array([a,1.*b]).T
  return [x , y] 

def qammod2(m,size,trials):
  """
  m - highest integer for the randint
  """
  y = np.random.randint(m,size=(trials,size))
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  #x = a + 1.*1j*b
  # lets hold x in a dp array([real,imag])
  #x = np.array([a,1.*b])
  x = np.dstack((a,b))
  return [x , y]                        

def channel2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  
  [x,y] = qammod2(m,sequenceLen,trials)
  
  h = [1., 0.5, 0.3, -0.13, 0.2, 0, 0.4]
  #h = np.random.randn(1,chanelLen)
  #h = [1]
  #print(x.shape)
#   xh = np.empty((sequenceLen+len(h)-1,2))
#   xh[:,0] = np.convolve(x[:,0],h)   
#   xh[:,1] = np.convolve(x[:,1],h)   
  #xh = convolve1d(x,h,axis = 1)
  xh = np.apply_along_axis(lambda m: np.convolve(m,h[0:chanelLen],mode="full"),axis = 1,arr=x)
  #print(xh.shape)
  SNR = 10**(SNRdB/10)
  sym_noise_pow = np.var(xh)/SNR
  sym_noise_scale_fact = np.sqrt(sym_noise_pow/2)
  if(Noise):
    AWGN = sym_noise_scale_fact*np.random.normal(1, size=xh.shape) 
    xh = xh + AWGN
  #return [xhN.T , np.repeat(y,2) ]
  return [xh , y ]
  
                        
data={}
def generateData(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  [data['x_train'],data['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_val'],data['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_test'],data['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen) 
                        
data2={}
def generateData2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  [data2['x_train'],data2['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data2['x_val'],data2['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data2['x_test'],data2['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)

In [0]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [0]:
[a,b]= unison_shuffled_copies(arr,arr1)

In [27]:
# Visulizing the data

m = 4         # order of modulation
#seqLen = 64    # length of one trial must be square powers of 2 to make sense see below 
trials = 10   # number of trials 
Noise = 1
SNRdB = 20
chanelLen = 1


x = 1
square_power_of_two = 2**(2*x)
seqLen = square_power_of_two  
print(seqLen)

generateData(4,seqLen,trials,Noise,SNRdB,chanelLen)
print(data['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data['y_train'].shape) # labels

generateData2(16,seqLen,trials,Noise,SNRdB,chanelLen)
print(data2['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data2['y_train'].shape) # labels
#print(data['x_train']) # so u can see the data 

# tmp = (data['x_train'][:,:-1].reshape(trials,seqLen))

# #plot 1
# plt.scatter(np.real(tmp[0][:]),np.imag(tmp[0][:]))   
#                                          # I only plot the first 10 points from 
#                                          # the sequence because as time goes on 
#                                          # the data becomes less dependent on the
#                                          # previous symbols. Even 10 is a lot. 
# plt.title('4-QAM Mod With Multi-Channel AWGN plot 1')
# plt.show()

#plot 2
# Just a pure 16 qam with 1 point on each constalation point 
# clearly we have 16 classes
# [xtmp,ytmp]=  qammod(16,data['y_train'])
# plt.scatter(xtmp[:,0],xtmp[:,1])
# plt.title('16-QAM Mod plot 2')
# plt.show()



4
(10, 4, 2)
(10, 4)
(10, 4, 2)
(10, 4)


In [61]:
data2['x_val']

(5, 4, 2)

In [0]:
newData2 = [data2['x_val'][i].flatten() for i  in range(5) ]

In [68]:
np.array(newData2)

array([[-0.80072411, -1.02612589, -0.76338581, -0.8388302 , -0.79342653,
         2.98834246,  2.93220043,  1.14278699],
       [ 3.05717389,  1.39868647, -0.79451396, -0.571688  , -0.79468801,
         3.23303537,  0.96992143, -0.85566655],
       [ 1.09653957,  0.83857989, -1.05357182, -0.76242245,  1.02772825,
         1.24770439, -0.92826459, -0.82873502],
       [ 3.39282965,  1.04548428,  3.25984904, -2.78916236, -0.99271701,
        -2.78900573, -0.72032754,  1.03350407],
       [-2.82252532,  3.36541624, -2.61673912, -2.78987202, -0.9762244 ,
         0.99284029, -2.71445683, -1.02343636]])

In [58]:
A =data2['x_val']
B = data['x_val']
D = np.concatenate((A,B),axis=0)
D

array([[[-0.80072411, -1.02612589],
        [-0.76338581, -0.8388302 ],
        [-0.79342653,  2.98834246],
        [ 2.93220043,  1.14278699]],

       [[ 3.05717389,  1.39868647],
        [-0.79451396, -0.571688  ],
        [-0.79468801,  3.23303537],
        [ 0.96992143, -0.85566655]],

       [[ 1.09653957,  0.83857989],
        [-1.05357182, -0.76242245],
        [ 1.02772825,  1.24770439],
        [-0.92826459, -0.82873502]],

       [[ 3.39282965,  1.04548428],
        [ 3.25984904, -2.78916236],
        [-0.99271701, -2.78900573],
        [-0.72032754,  1.03350407]],

       [[-2.82252532,  3.36541624],
        [-2.61673912, -2.78987202],
        [-0.9762244 ,  0.99284029],
        [-2.71445683, -1.02343636]],

       [[-0.97709193,  1.21857591],
        [ 1.1619601 ,  1.16476444],
        [-0.89085898, -1.03737182],
        [ 1.04617769, -0.96070812]],

       [[ 0.9589692 , -0.93347771],
        [ 1.00434018, -0.85368082],
        [-0.87016215,  1.00795955],
        [-0.8966

In [60]:
data2['y_val'][0]

array([ 6,  6,  4, 13])

In [37]:
np.stack((data2['x_val'],data['x_val']),axis=0).shape


(2, 5, 4, 2)

In [19]:
arr1 = np.arange(9).reshape((3, 3))
arr1

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [20]:

np.random.shuffle(arr1)
arr1

array([[3, 4, 5],
       [6, 7, 8],
       [0, 1, 2]])

In [0]:

# Some keras stuffs
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Reshape, Permute
from keras import regularizers
import keras
ykeys = {'y_train','y_val','y_test'}
xkeys = {'x_train','x_val','x_test'}

  
y_train = keras.utils.to_categorical(data['y_train'] , num_classes=m)
y_val = keras.utils.to_categorical(data['y_val'] , num_classes=m)
y_test = keras.utils.to_categorical(data['y_test'] , num_classes=m)

print(data['x_train'].shape)
print(data['x_test'].shape)
print(y_train.shape)
print(data['x_val'].shape)


default_data = {
  'x_train': data['x_train'],
  'x_val': data['x_val'],
  'x_test': data['x_test'],
  'y_train': y_train,
  'y_val': y_val,
  'y_test': y_test,
}


In [0]:
# QAM-4
from keras.layers import Reshape,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers

modCNN = Sequential()
modCNN.add(Conv1D(2,kernel_size=(3), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(2,kernel_size=(3)))
modCNN.add(BatchNormalization())
modCNN.add(Flatten())
modCNN.add(Dense(seqLen*m,bias_regularizer=regularizers.l2(0.005)))
modCNN.add(Reshape(target_shape = (seqLen,m)))
modCNN.add(BatchNormalization())
#modCNN.add(Dropout(0.4))
modCNN.add(Softmax(axis=2))
try_model(modCNN, default_data, 'conv_lstmx', epochs=3)
